In [49]:
import numpy as np
import pandas as pd

print np.__version__
print pd.__version__

1.9.2
0.16.1


In [50]:
 pd.set_option('display.notebook_repr_html',False)

## group by

### aggregated data
* events are added per (member, org)

In [51]:
data = {'member_id': [1,3,3,4,5,1,1],
        'org_id': [1,2,2,1,1,1,2],
        'event': ['up']*4 + ['down']*3,
        'score': [5,6,6,4,6,2,7],
        'hits': [4,3,1,2,2,8,1]}

df = pd.DataFrame(data)
df

  event  hits  member_id  org_id  score
0    up     4          1       1      5
1    up     3          3       2      6
2    up     1          3       2      6
3    up     2          4       1      4
4  down     2          5       1      6
5  down     8          1       1      2
6  down     1          1       2      7

In [52]:
grouped = df.groupby(['member_id', 'org_id'])
grouped

In [53]:
g = df.groupby(['member_id', 'org_id'])
g.size().to_dict()

{(1, 1): 2, (1, 2): 1, (3, 2): 2, (4, 1): 1, (5, 1): 1}

In [54]:
grouped.get_group((1, 1))

  event  hits  member_id  org_id  score
0    up     4          1       1      5
5  down     8          1       1      2

keep events with highest score in (member, org) group

In [55]:
grouped.transform(max)

  event  hits  score
0    up     8      5
1    up     3      6
2    up     3      6
3    up     2      4
4  down     2      6
5    up     8      5
6  down     1      7

In [56]:
df[df['score'] == grouped.transform(max)['score']]

  event  hits  member_id  org_id  score
0    up     4          1       1      5
1    up     3          3       2      6
2    up     1          3       2      6
3    up     2          4       1      4
4  down     2          5       1      6
6  down     1          1       2      7

in case a solution is required that always picks exactly one
introduce a tie-breaker

In [57]:
the_rule = np.array([1,3,2,3])
the_rule[the_rule.argmax()] += 1
the_rule

array([1, 4, 2, 3])

In [58]:
def majority_rule(x):
    if len(x[x == x.max()]) == 1:
        # max is unique
        return x.max()
    else:
        # tie-break
        x.ix[x.argmax()] -= 1
        return x

grouped['score'].transform(majority_rule)

0    5
1    5
2    6
3    4
4    6
5    5
6    7
Name: score, dtype: int64

### TimeGrouper

raw data comes from an event stream
one row records a single event occured at a certain time

In [59]:
from datetime import datetime

data = {'timestamp': [datetime(2015, 7, 1, 9, 0, 0), datetime(2015, 7, 1, 9, 0, 29), datetime(2015, 7, 1, 9, 0, 7),
                      datetime(2015, 7, 1, 9, 1, 32),
                      datetime(2015, 7, 1, 9, 0, 13), datetime(2015, 7, 1, 9, 0, 23)],
        'member_id': [1, 1, 1, 1, 2, 2],
        'event': ['down', 'up', 'up', 'down', 'up', 'enterprise']}

df = pd.DataFrame(data)
df = df.set_index('timestamp')
df

                          event  member_id
timestamp                                 
2015-07-01 09:00:00        down          1
2015-07-01 09:00:29          up          1
2015-07-01 09:00:07          up          1
2015-07-01 09:01:32        down          1
2015-07-01 09:00:13          up          2
2015-07-01 09:00:23  enterprise          2

put all events of a member that occured within the same minute into a separate group

In [60]:
from pandas.tseries.offsets import Minute

tg = df.groupby([df.member_id, pd.TimeGrouper(Minute(1))])
for g, gf in tg:
    print gf.event

timestamp
2015-07-01 09:00:00    down
2015-07-01 09:00:29      up
2015-07-01 09:00:07      up
Name: event, dtype: object
timestamp
2015-07-01 09:01:32    down
Name: event, dtype: object
timestamp
2015-07-01 09:00:13            up
2015-07-01 09:00:23    enterprise
Name: event, dtype: object


count the events within a minute

In [61]:
tg_events = tg['event'].value_counts()
tg_events

member_id  timestamp                      
1          2015-07-01 09:00:00  up            2
                                down          1
           2015-07-01 09:01:00  down          1
2          2015-07-01 09:00:00  enterprise    1
                                up            1
dtype: int64

### unstack, reset_index

groupby returns a MultiIndex

In [62]:
tg_events.index

MultiIndex(levels=[[1, 2], [2015-07-01 09:00:00, 2015-07-01 09:01:00], [u'down', u'enterprise', u'up']],
           labels=[[0, 0, 0, 1, 1], [0, 0, 1, 0, 0], [2, 0, 0, 1, 2]],
           names=[u'member_id', u'timestamp', None])

unstack "deepest" level

In [63]:
tg_unstacked = tg_events.unstack().fillna(0)
tg_unstacked

                               down  enterprise  up
member_id timestamp                                
1         2015-07-01 09:00:00     1           0   2
          2015-07-01 09:01:00     1           0   0
2         2015-07-01 09:00:00     0           1   1

unstack one more level

In [64]:
tg_unstacked.unstack().fillna(0)

                         down                              enterprise  \
timestamp 2015-07-01 09:00:00 2015-07-01 09:01:00 2015-07-01 09:00:00   
member_id                                                               
1                           1                   1                   0   
2                           0                   0                   1   

                                               up                      
timestamp 2015-07-01 09:01:00 2015-07-01 09:00:00 2015-07-01 09:01:00  
member_id                                                              
1                           0                   2                   0  
2                           0                   1                   0  

"flatten" the MultiIndex to get (member, minute) counts of each event

In [65]:
tg_unstacked.reset_index()

   member_id           timestamp  down  enterprise  up
0          1 2015-07-01 09:00:00     1           0   2
1          1 2015-07-01 09:01:00     1           0   0
2          2 2015-07-01 09:00:00     0           1   1

* dataframe with the unique member_id
* DataFrameGroupBy convert to DataFrame
```
SELECT member_id
FROM multi_key
GROUP BY member_id
```

In [66]:
multi_key = pd.DataFrame({'member_id': [4, 5, 4, 5],
                          'organization_id': [3,3,4,3]})

counted = multi_key.groupby(['member_id', 'organization_id']).count()
type(counted)

pandas.core.frame.DataFrame

it is a multi-index so this is not going to work:

In [67]:
try:
    counted[['member_id', 'organization_id']]
except:
    print 'Nice try.'

Nice try.
